# 문자 단위 RNN
  - 모든 시점의 입력에 대해서 모든 시점에 대해서 출력을 하는 다대다 RNN 구현
  - 문자 단위란?
    - word-level이 아니라 character-level로 구현

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

### 문자 시퀀스 apple을 받으면 -> pple!를 출력하는 RNN
  - input data와 label data에 대해서 문자 집합을 만든다. (중복 제거)

In [ ]:
#문자 집합을 만든다
input_str = 'apple'
label_str = 'pple!'
char_vocab = sorted(list(set(input_str+label_str))) #set 중복 제거
print(char_vocab)

['!', 'a', 'e', 'l', 'p']


In [ ]:
vocab_size = len(char_vocab)
print(f'문자 집합의 크기:{vocab_size}')

문자 집합의 크기:5


In [ ]:
#원 핫 벡터를 사용할 것이므로 입력의 크기 = 문자 집합의 크기
input_size = vocab_size
hidden_size = 5
output_size = 5
lr = 0.1

In [ ]:
#문자에 고유한 정수 인덱스 부여
char_to_index = dict((c,i) for i,c in enumerate(char_vocab))
print(char_to_index)

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}


In [ ]:
index_to_char = {} #나중에 예측 결과를 다시 문자 시퀀스로 보기 위해서 반대로도 만들어 놓음
for key,value in char_to_index.items():
  index_to_char[value] = key
print(index_to_char)

{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


In [ ]:
print(input_str)

apple


In [ ]:
char_to_index['a']

1

In [ ]:
x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]
print(x_data) #a,p,p,l,e
print(y_data) #p,p,l,e,!

[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]


In [ ]:
#파이토치의 nn.RNN()은 기본적으로 3차원 텐서를 입력 받음. 그렇기 떄문에 배치 차원을 추가해준다
#배치 차원 추가
#텐서 연산인 unsqueeze(0)를 통해 해결할 수도 있음

x_data = [x_data]
y_data = [y_data]
print(x_data) #a, p, p, l, e
print(y_data) #p, p, l, e, !

[[1, 4, 4, 3, 2]]
[[4, 4, 3, 2, 0]]


### expand_dims로도 차원을 추가해줄 수 있음

In [ ]:
x_data_unsqueeze = np.expand_dims(x_data,0)
print(x_data_unsqueeze)

[[[1 4 4 3 2]]]


In [ ]:
torch.LongTensor(x_data).unsqueeze(0)

tensor([[[1, 4, 4, 3, 2]]])

In [ ]:
x_one_hot = [np.eye(vocab_size)[x] for x in x_data] #총 행렬의 크기 5x5로 만들어라 
print(x_one_hot)

[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]


In [ ]:
np.eye(vocab_size)

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [ ]:
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

print(f'훈련 데이터의 크기:{X.shape}')
print(f'레이블의 크기:{Y.shape}')

훈련 데이터의 크기:torch.Size([1, 5, 5])
레이블의 크기:torch.Size([1, 5])


In [ ]:
class Net(nn.Module):
  def __init__(self,input_size,hidden_size,output_size):
    super(Net,self).__init__()
    self.rnn = nn.RNN(input_size,hidden_size,batch_first=True) #RNN 셀 구현

    self.fc = nn.Linear(hidden_size,output_size,bias=True) #출력층 구현

  def forward(self,x):
    x,_status = self.rnn(x)
    x = self.fc(x)
    return x

In [ ]:
net = Net(input_size,hidden_size,output_size)

In [ ]:
criterion = nn.CrossEntropyLoss()
oprimizer = optim.Adam(net.parameters(),lr)

outputs = net(X)
print(outputs)
print('')
print(outputs.shape) #배치 차원, 시점(timesteps), 출력의 크기 

tensor([[[ 0.2935,  0.3544,  0.3451,  0.5131,  0.3802],
         [-0.1107,  0.3920,  0.3779,  0.5646,  0.2775],
         [-0.0144,  0.3896,  0.4192,  0.5535,  0.3788],
         [-0.0586,  0.6031,  0.2736,  0.2566,  0.2084],
         [-0.0397,  0.5191,  0.4932,  0.5006,  0.4519]]],
       grad_fn=<AddBackward0>)

torch.Size([1, 5, 5])


In [ ]:
print(outputs.view(-1,input_size).shape) #2차원 텐서로 변환 

torch.Size([5, 5])


In [ ]:
print(Y.shape)
print(Y.view(-1).shape) #정확도를 측정할때는 이걸 펼쳐서 계산할 예정이니까
print(Y)

torch.Size([1, 5])
torch.Size([5])
tensor([[4, 4, 3, 2, 0]])


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(),lr)

In [ ]:
for i in range(10):
  optimizer.zero_grad()
  outputs = net(X)
  loss = criterion(outputs.view(-1,input_size), Y.view(-1)) #view를 하는 이유 Batch차원 제거를 위해

  loss.backward()
  optimizer.step()

  #모델이 실제 어떻게 예측했는지를 확인하기 위한 코드
  result = outputs.data.numpy().argmax(axis=2)
  print(result)
  result_str = ''.join([index_to_char[c] for c in np.squeeze(result)])
  print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

[[3 3 3 1 1]]
0 loss:  1.6705913543701172 prediction:  [[3 3 3 1 1]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  lllaa
[[4 4 3 4 4]]
1 loss:  1.4243745803833008 prediction:  [[4 4 3 4 4]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pplpp
[[4 4 3 4 4]]
2 loss:  1.2227948904037476 prediction:  [[4 4 3 4 4]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pplpp
[[4 4 3 2 4]]
3 loss:  1.0377991199493408 prediction:  [[4 4 3 2 4]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pplep
[[4 4 3 2 4]]
4 loss:  0.858181357383728 prediction:  [[4 4 3 2 4]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pplep
[[4 4 3 2 4]]
5 loss:  0.7077634334564209 prediction:  [[4 4 3 2 4]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pplep
[[4 4 3 2 4]]
6 loss:  0.570701539516449 prediction:  [[4 4 3 2 4]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pplep
[[4 4 3 2 0]]
7 loss:  0.43255677819252014 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
[[4 4 3 2 0]]
8 loss:  0.3455711007118225 predict

## 더 많은 문장으로 했을때는?

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim 

In [ ]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

In [ ]:
len(sentence)

180

In [ ]:
char_set = list(set(sentence)) #중복을 제거한 문자 집합
char_dic = {c: i for i, c in enumerate(char_set)} #각 문자에 정수 인코딩
print(char_dic)

{'i': 0, 'a': 1, 'w': 2, 'r': 3, '.': 4, 'u': 5, 'y': 6, "'": 7, 'k': 8, ',': 9, 'g': 10, 'c': 11, 'n': 12, 'h': 13, 'b': 14, 's': 15, 'd': 16, 'o': 17, 'l': 18, 'e': 19, 'f': 20, 't': 21, 'p': 22, ' ': 23, 'm': 24}


In [ ]:
dic_size = len(char_dic)
print(f'문자 집합의 크기:{dic_size}')

문자 집합의 크기:25


In [ ]:
hidden_size = dic_size
sequence_length = 10 #임의 숫자 
lr = 0.1

In [ ]:
#10의 단위로 샘플들을 잘라서 데이터를 만듬
x_data = []
y_data = []

for i in range(0,len(sentence) - sequence_length):
  x_str = sentence[i: i + sequence_length]
  y_str = sentence[i+1: i + sequence_length + 1]
  print(i, x_str, '->', y_str)

  x_data.append([char_dic[c] for c in x_str]) # x str to index
  y_data.append([char_dic[c] for c in y_str]) # y str to index

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [ ]:
print(x_data[0]) #if you wan에 해당
print(y_data[0]) # f you want에 해당
#한 칸씩 쉬프트로 된 시퀀스 
#<sos>?

[0, 20, 23, 6, 17, 5, 23, 2, 1, 12]
[20, 23, 6, 17, 5, 23, 2, 1, 12, 21]


In [ ]:
x_one_hot = [np.eye(dic_size)[x] for x in x_data] #x 데이터는 원-핫 인코딩
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [ ]:
print(f'훈련 데이터의 크기:{X.shape}')
print(f'레이블의 크기:{Y.shape}')

훈련 데이터의 크기:torch.Size([170, 10, 25])
레이블의 크기:torch.Size([170, 10])


In [ ]:
print(X[0])

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers): # 현재 hidden_size는 dic_size와 같음.
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [ ]:
net = Net(dic_size, hidden_size, 2) # 이번에는 층을 두 개 쌓습니다.

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr)

In [ ]:
outputs = net(X)
print(outputs.shape) # 3차원 텐서

torch.Size([170, 10, 25])


In [ ]:
print(outputs.view(-1, dic_size).shape) # 2차원 텐서로 변환.

torch.Size([1700, 25])


In [ ]:
for i in range(30):
    optimizer.zero_grad()
    outputs = net(X) # (170, 10, 25) 크기를 가진 텐서를 매 에포크마다 모델의 입력으로 사용
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    # results의 텐서 크기는 (170, 10)
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오지만
            predict_str += ''.join([char_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += char_set[result[-1]]

    print(predict_str)

esbssssssssssssssssbsssssssssssssbsssssssbssssbsbssssssbbsssssssbbbssssssssssssssssbssssssssssbssssssbssssssssbbssssssssbbssssssbssbbssssssbsssssssssbsssbbbssssssssbssbssssssbssss
                                                                                                                                                                                   
 h,hdhnhnhh,hnhnp,,hnhtp,,,eshnhtuhhh,m,h,h,mmm,edhn,mem,m,h,,,dmnheh,,,,,,hehnhn,hhtuh,hemnhd,mnhnh,,,,m,,hnhtutuheh,heh,m,hehem,,m,,mh,,,emnhdhhnhemm,h,m,sh,,m,m,h,heh.hnhnh,,,,
  hot tot o ouh  t o otow t ton t t tot   t tgot ho  h o s o hh   o oe     tuo onot t t t  oo  gto  ho  woton t t t o   t r o  he      n goo  to hton t t t t t w t  ht ouoton     
    r t t    t t t  t t t e t t t t e t      t t t          t t  n   t        t t t t    n o    t   t   w t t t t t   t t t    t         en    o  t t t t e t t e t      t t t     
mmtorh lntltl l toeotleoe etleele ltleleelle leo et etoetle elnnln e dntl t'eoe elb et nletllenle er

## Embedding 사용!!
  - embedding layer를 사용

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
sentence = "Repeat is the best medicine for memory".split()
vocab = list(set(sentence))
print(vocab)
#우리가 만들 RNN은 "Repeat is the best medicine for"을 입력 받으면 "is the best medicine for memory"를 출력하는 RNN

['for', 'Repeat', 'best', 'the', 'is', 'memory', 'medicine']


In [ ]:
word2index = {tkn:i for i,tkn in enumerate(vocab,1)} #단어에 고유한 정수 부여
word2index['<unk>']=0
print(word2index)

{'for': 1, 'Repeat': 2, 'best': 3, 'the': 4, 'is': 5, 'memory': 6, 'medicine': 7, '<unk>': 0}


In [ ]:
index2word = {v: k for k, v in word2index.items()}
print(index2word)

{1: 'for', 2: 'Repeat', 3: 'best', 4: 'the', 5: 'is', 6: 'memory', 7: 'medicine', 0: '<unk>'}


In [ ]:
#우리가 사용할 word2index단어장
def build_data(sentence,word2index):
  encoded = [word2index[token] for token in sentence] #각 문자를 정수로 변환
  input_seq, label_seq = encoded[:-1],encoded[1:] #입력 시퀀스와 레이블 시퀀스를 분리
  input_seq = torch.LongTensor(input_seq).unsqueeze(0) #배치 차원 추가
  label_seq = torch.LongTensor(label_seq).unsqueeze(0) #배치 차원 추가
  return input_seq,label_seq

In [ ]:
X,Y = build_data(sentence,word2index)
print(X)
print(Y)

tensor([[2, 5, 4, 3, 7, 1]])
tensor([[5, 4, 3, 7, 1, 6]])


In [ ]:
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, # 워드 임베딩
                                            embedding_dim=input_size)
        self.rnn_layer = nn.RNN(input_size, hidden_size, 
                                batch_first=batch_first)
        self.linear = nn.Linear(hidden_size, vocab_size) 

    def forward(self, x):
        # 1. 임베딩 층
        # 크기변화: (배치 크기, 시퀀스 길이) => (배치 크기, 시퀀스 길이, 임베딩 차원)
        output = self.embedding_layer(x)
        # 2. RNN 층
        # 크기변화: (배치 크기, 시퀀스 길이, 임베딩 차원)
        # => output (배치 크기, 시퀀스 길이, 은닉층 크기), hidden (1, 배치 크기, 은닉층 크기)
        output, hidden = self.rnn_layer(output)
        # 3. 최종 출력층
        # 크기변화: (배치 크기, 시퀀스 길이, 은닉층 크기) => (배치 크기, 시퀀스 길이, 단어장 크기)
        output = self.linear(output)
        # 4. view를 통해서 배치 차원 제거
        # 크기변화: (배치 크기, 시퀀스 길이, 단어장 크기) => (배치 크기*시퀀스 길이, 단어장 크기)
        return output.view(-1, output.size(2))

In [ ]:
vocab_size = len(word2index) 
input_size = 5      
hidden_size = 20   

In [ ]:
model = Net(vocab_size, input_size, hidden_size, batch_first=True)
loss_function = nn.CrossEntropyLoss() 
optimizer = optim.Adam(params=model.parameters())

In [ ]:
#(시퀀스의 길이, 은닉층의 크기)
output = model(X)
print(output)
print('')
print(output.shape)

tensor([[ 0.2843,  0.4345, -0.1295,  0.0404, -0.1824, -0.0354, -0.0799, -0.0787],
        [ 0.4519,  0.3138, -0.0463, -0.2250, -0.1907, -0.2845, -0.1057,  0.3123],
        [-0.0616,  0.5515, -0.4119,  0.4048, -0.2292, -0.1598, -0.3653, -0.4435],
        [ 0.1207,  0.5257, -0.1004, -0.0414, -0.3587, -0.1495, -0.0245, -0.0326],
        [ 0.5649,  0.2861, -0.1643, -0.0426, -0.1589, -0.2638, -0.2193,  0.0693],
        [ 0.0561,  0.0034,  0.1165, -0.0520, -0.1553, -0.4532, -0.1832, -0.0336]],
       grad_fn=<ViewBackward>)

torch.Size([6, 8])


In [ ]:
decode = lambda y: [index2word.get(x) for x in y]

In [ ]:
for step in range(201):
    optimizer.zero_grad()
    output = model(X)
    loss = loss_function(output, Y.view(-1))
    loss.backward()
    optimizer.step()
    
    # 기록
    if step % 40 == 0:
        print("[{:02d}/201] {:.4f} ".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(["Repeat"] + decode(pred)))
        print()

[01/201] 2.0533 
Repeat for <unk> for for <unk> Repeat

[41/201] 1.3735 
Repeat is medicine best medicine for memory

[81/201] 0.8015 
Repeat is the best medicine for memory

[121/201] 0.3995 
Repeat is the best medicine for memory

[161/201] 0.2091 
Repeat is the best medicine for memory

[201/201] 0.1274 
Repeat is the best medicine for memory



Reference: https://wikidocs.net/64765 를 참조하였습니다.


### word based or character based?
  - word based: higher accuracy and lower computational cost than character based LMs
  - however, character based RNN LMs better model languages with a rich morphology(형태론) such as Finish, Turkish, Russian etc. Using word based RNN LMs to model such languages is difficult if possible at all and is not advised
  - out of vocabulary word, because can only handle those seen words

형태론 - 단어의 어형(語形) 변화를 다루는 문법의 한 분야이다.

한 언어에서 형태소들이 결합하여 낱말을 형성하는 체계 또는 규칙으로, 형태소(morpheme) 및 낱말을 기본 단위로 한다.